In [2]:
import os
import numpy as np

import pandas as pd

In [3]:
data = pd.read_csv("./avatars_echOvid_train_X.csv")

display(data)


,id,Nom du centre,age,Sexe,Lieu de provenance du patient,Echographiste,Tabagisme actif,BPCO,Asthme,Autre antecedent respiratoire,...,Zone anterieure droite haut,Zone anterieure droite bas,Zone anterieure gauche haut,Zone anterieure gauche bas,Zone posterieure droite haut,Zone posterieure droite bas,Zone posterieure gauche haut,Zone posterieure gauche bas,Oxygenotherapie,Outcome J28
0,1,Paris-APHP - Cochin,91,Masculin,Domicile,Forme pour l'epidemie,Non,Non,Non,Non,...,Stade 2,Stade 2,Stade 1,Stade 2,Stade 2,Stade 2,Stade 2,Stade 2,NaN,Death
1,109,Paris-APHP - Hotel Dieu,45,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Moderee,Back home
2,46,Melun - CH Marc Jacquet,76,Feminin,EHPAD,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 1,Stade 0,Stade 1,Stade 2,Stade 1,Stade 2,Assistance respiratoire,Hospitalization
3,247,Paris-APHP - Hotel Dieu,55,Feminin,Hopital,Experience d'echographie,Oui,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
4,195,Paris-APHP - Hotel Dieu,42,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,218,Paris-APHP - Cochin,34,Masculin,Domicile,Forme pour l'epidemie,Oui,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
258,199,Melun - CH Marc Jacquet,58,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 1,Stade 1,Stade 0,Stade 0,Stade 1,Stade 1,Stade 0,Stade 1,Assistance respiratoire,Hospitalization
259,117,Paris-APHP - Cochin,45,Feminin,Domicile,Expert,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
260,201,Paris-APHP - Cochin,46,Feminin,Domicile,Expert,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home


In [4]:
data.dtypes

id                                                                            int64
Nom du centre                                                                object
age                                                                           int64
Sexe                                                                         object
Lieu de provenance du patient                                                object
Echographiste                                                                object
Tabagisme actif                                                              object
BPCO                                                                         object
Asthme                                                                       object
Autre antecedent respiratoire                                                object
Hypertension arterielle                                                      object
Cardiopathie ischemique                                                     

In [5]:
#feature_cols = []
feature_cols = ['Zone anterieure droite bas', 'Zone anterieure droite haut','Zone anterieure gauche bas', 'Zone anterieure gauche haut', 'Zone posterieure droite bas', 'Zone posterieure droite haut','Zone posterieure gauche bas', 'Zone posterieure gauche haut', 'Date de debut de la symptomatologie', 'Diabetes']

def dt64_to_float(dt64):
  year = dt64.astype('M8[Y]')
  days = (dt64 - year).astype('timedelta64[D]')
  year_next = year + np.timedelta64(1, 'Y')
  days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
  dt_float = 1970 + year.astype(float) + days / (days_of_year)
  return dt_float

data['Date de debut de la symptomatologie'] = dt64_to_float(data['Date de debut de la symptomatologie'].to_numpy())
x = data['date_float'].values

# Change date from object to datetime
#data['Date de debut de la symptomatologie'] = pd.to_datetime(data['Date de debut de la symptomatologie'])

def convert(x):
  ret_value = x
  if x == 'Oui':
    ret_value= 0
  elif x == 'Non':
    ret_value= 1
  elif type(x) == str and x.split(' ')[0] == 'Stade':
    ret_value= int(x[-1])
  return ret_value

display(data[feature_cols])
X =data[feature_cols].apply(lambda value: value.map(lambda x: convert(x)))
display(X)


y = data['Outcome J28']

ValueError: Could not convert object to NumPy datetime

In [139]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [140]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# instantiate model
logreg = RandomForestClassifier()

# fit model
logreg.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [73]:
# make class predictions for the testing set
y_pred_class = logreg.predict(X_test)

In [74]:

from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.8333333333333334


In [76]:
y_test.value_counts()


Back home              39
Hospitalization        23
Death                   2
Intensive care unit     2
Name: Outcome J28, dtype: int64

In [77]:
print(metrics.confusion_matrix(y_test, y_pred_class))


[[34  0  5  0]
 [ 0  0  0  2]
 [ 3  0 20  0]
 [ 0  0  1  1]]


In [78]:
y_test

194          Back home
153    Hospitalization
145          Back home
8            Back home
111          Back home
            ...       
184          Back home
15     Hospitalization
178          Back home
96           Back home
18               Death
Name: Outcome J28, Length: 66, dtype: object

In [80]:
test_data = pd.read_csv("./avatars_echOvid_test_X.csv")

X_test =test_data[feature_cols].apply(lambda value: value.map(lambda x: int(x[-1])))

y_pred_class = logreg.predict(X_test)

result_dataFrame =pd.DataFrame({'id':test_data['id'],'prediction':y_pred_class})
result_dataFrame = result_dataFrame.set_index('id')

display(result_dataFrame)
result_dataFrame.to_csv("./result.csv", sep =';')


,prediction
id,
237,Back home
236,Back home
253,Back home
92,Back home
53,Hospitalization
...,...
151,Hospitalization
39,Back home
313,Back home
